In [1]:
import sys
sys.path.append("../")
import os
data_path = "./txt"

In [2]:
import json
import requests
import re
import numpy as np
import pickle
import gzip
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI # API v1.2
from bs4 import BeautifulSoup

with open("../openai_key.txt", "r") as f:
    openai_key = f.read().strip()
    openai_client = OpenAI(api_key=openai_key)

In [3]:
def get_all_files_in_folder(folder_path):
    file_list = []
    for root, directories, files in os.walk(folder_path):
        for filename in files:
            if filename=="README.md": continue
            if not filename.endswith(".html"): continue
            file_list.append(os.path.join(root, filename))
    return file_list

file_list = get_all_files_in_folder(data_path)
file_list = sorted(file_list)
print(len(file_list))

620


In [4]:
def extract_titles(files, out_file="titles.json"):
    """提取每个文档的标题，并按照编号整理到一个json文件中"""
    dict1 = dict()
    for id, in_file in enumerate(tqdm(files)):
        file_name = in_file.split("/")[-1].split(".")[0]
        #with open(in_file, "r") as f:
        #title = f.readlines()[0]
        dict1[file_name] = file_name
    with open(out_file, "w") as f:
        json.dump(dict1, f, ensure_ascii=False, indent=4)

def load_titles(file="titles.json"):
    """读取标题文件"""
    with open(file, "r") as f:
        dict1 = json.load(f)
    return dict1

extract_titles(file_list, out_file="titles.json")
dict1 = load_titles(file="titles.json")
print(len(dict1))
print(dict1)

100%|█████████████████████████████████████████████████████████████████████████████| 620/620 [00:00<00:00, 277265.00it/s]

620
{'20190408 陈医师访谈刘仲敬第31集 - 刘仲敬文稿站 (24_07_2024 6_47_15 PM)-checkpoint': '20190408 陈医师访谈刘仲敬第31集 - 刘仲敬文稿站 (24_07_2024 6_47_15 PM)-checkpoint', '20190408 陈医师访谈刘仲敬第31集 - 刘仲敬文稿站 (24_07_2024 6_47_15 PM)': '20190408 陈医师访谈刘仲敬第31集 - 刘仲敬文稿站 (24_07_2024 6_47_15 PM)', '20190414 陈医师访谈刘仲敬第32集 - 刘仲敬文稿站 (24_07_2024 6_47_18 PM)': '20190414 陈医师访谈刘仲敬第32集 - 刘仲敬文稿站 (24_07_2024 6_47_18 PM)', '20190422 陈医师访谈刘仲敬第33集 - 刘仲敬文稿站 (24_07_2024 6_47_20 PM)': '20190422 陈医师访谈刘仲敬第33集 - 刘仲敬文稿站 (24_07_2024 6_47_20 PM)', '20190426 陈医师访谈刘仲敬第34集 - 刘仲敬文稿站 (24_07_2024 6_47_22 PM)': '20190426 陈医师访谈刘仲敬第34集 - 刘仲敬文稿站 (24_07_2024 6_47_22 PM)', '20190430 近代人物点评：总序 - 刘仲敬文稿站 (26_08_2024 7_06_38 AM)': '20190430 近代人物点评：总序 - 刘仲敬文稿站 (26_08_2024 7_06_38 AM)', '20190501 近代人物点评1：晚清“王熙凤”—慈禧 - 刘仲敬文稿站 (26_08_2024 7_06_40 AM)': '20190501 近代人物点评1：晚清“王熙凤”—慈禧 - 刘仲敬文稿站 (26_08_2024 7_06_40 AM)', '20190502 近代人物点评2：富有责任感的袁世凯 - 刘仲敬文稿站 (26_08_2024 7_06_42 AM)': '20190502 近代人物点评2：富有责任感的袁世凯 - 刘仲敬文稿站 (26_08_2024 7_06_42 AM)', '20190503 近代人物点评3：自以为和被以为重要的李

In [14]:
def util_collect_paragraph(data):
    l2 = []
    soup = BeautifulSoup(data, 'html.parser')
    l1 = soup.find_all('p')
    for i, x in enumerate(l1):
        x = x.get_text()
        if len(x)<200: continue
        l2.append(x)
    return l2

def load_data_to_paragraphs(file):
    """ 将长文档分解成1000字以内短文档. 
    因为openai sentence embedding ada 002 8000 input token, 最大2000汉字
    但是逼近2000后语义编码效果会下降
    """
    with open(file, "r") as f:
        data = f.read()
    
    l2 = util_collect_paragraph(data)
    data = data.replace('<p>', 'TEMP_P_TAG')
    data = data.replace('</p>', '<p>')
    data = data.replace('TEMP_P_TAG', '</p>')
    l2 += util_collect_paragraph(data)
    
    data = " ".join(l2)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    txts = text_splitter.split_text(data)
    return txts

def sentence_embedding_batch(txts, file_name):
    """将list of text编码为sentence embedding 1536维"""
    l1 = []
    model="text-embedding-3-small"
    #print(txts)
    l2 = openai_client.embeddings.create(input=txts, model=model).data
    embs = [x.embedding for x in l2]
    for i, txt in enumerate(txts):
        label = f"{file_name}-{i}"
        emb = embs[i]
        l1.append((label, txt, emb))
    return l1

def encoding_file(in_file, output_dir, skip_exist=True):
    """将文档.txt文件分割并编码为sentence embedding，压缩保存为同名.npz文件"""
    file_name = os.path.basename(in_file).split(".")[0]
    out_file = os.path.join(output_dir, file_name+".npz")
    if skip_exist: 
        if os.path.isfile(out_file): return
    txts = load_data_to_paragraphs(in_file)
    txts = [x for x in txts if len(x)>50]
    try:
        if len(txts)==0: 
            print(f"skip empty file: {file_name}")
            return
        packs = sentence_embedding_batch(txts, file_name)
        serialized_data = pickle.dumps(packs)
        compressed_data = gzip.compress(serialized_data)
        with open(out_file, "wb") as file:
            file.write(compressed_data)
    except Exception as e:
        print(file_name)
        print(txts[0])
        print(e)

def encoding_files(files, output_dir="./emb"):
    """批量将文件夹下txt文件编码为同名 embedding文件，2866个文件需要 openai 3美元"""
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    for i, in_file in enumerate(tqdm(files)):
        encoding_file(in_file, output_dir)

def decoding_file(file):
    with open(file, "rb") as f:
        compressed_data = f.read()
    decompressed_data = gzip.decompress(compressed_data)
    l1 = pickle.loads(decompressed_data)
    return l1

In [15]:
encoding_files(file_list, output_dir = "./emb") # 编码，保存到文件

100%|█████████████████████████████████████████████████████████████████████████████████| 620/620 [00:12<00:00, 48.81it/s]

从华夏到中国 - 刘仲敬文稿站 (31_07_2024 1_16_20 AM)
壹 从华夏到中国
1、海岱武装殖民
2、高克逍遥河上
3、秦为野兽
4、县而未封
5、项王耻渡乌江
6、韩侯甘居胯下——评《史记》
7、汝颖之士利如锥——评《后汉书》
8、儒化家族五百年
9、无恒产者：始于宾师，止于私属——评《东观汉记》
10、“三代以下风俗之美，无尚于东京者”——评《后汉书》
11、“孔融鲁国男子”
12、儒生右倾翻案风
13、清谈，清流之余绪——评《三国志》
14、唐承北朝方兴之气——评《北齐书》
15、行仁义而国亡，天何为而此醉——评《梁书》
16、畀无疆之孑遗，阖秦望之川薮——评《陈书》
17、南蛮校尉诸问题——评《南史》
18、复汉魏衣冠，用东齐之法——评《隋书》
19、赵魏多名儒，河朔尽夷狄——评《旧唐书》
20、地虽中州，风俗过于蛮夷
21、不出百年，天下民力竭矣——评《宋史》
22、武后之智，仁宗之仁
23、宋祖之军事济贫院
24、岳家军之商道
25、贾少师打土豪分田地
26、此地可称鬼国
27、盐、钞之弊，极于斯矣——评《元史》
28、“别了，马可波罗”之后
29、明祖之新三权分立
30、挑动运河鲁西荒
31、两淮官盐流毒天下
32、琦善、林则徐之治河策——评《清代水利与区域社会》
33、西蜀巉岩云迷鸟道，东粤碧海日警狼烟
34、汉传佛教无产化
35、李翰林守护国体
36、陈国瑞捍卫主权
37、汉恩自浅胡自深——评《编户齐民》 贰 万国与万民
1、神社
2、乌克兰及顿河哥萨克极简史
3、两个日耳曼
4、灵魂分裂的国家
5、文明的秋天
6、殖民主义
7、德奥和平统一及其敬酒罚酒
8、格莱斯顿之人权外交、价值观外交
9、西耶斯、贡斯当拉美转世记
10、上下秘鲁印第安贡赋问题
11、海地革命、小农结构、共和不立宪极简史
12、罗斯——书《秦晖：“近卫军”与“彼得”间的徘徊——索尔仁尼琴与俄国的“分裂教派”传统》后
13、华沙陷没，俄折我先锋!
14、恺撒及其幸运
15、面包不在内么？
16、路易十一长驻图尔
17、人君自食其邑
18、十七世纪常备军民兵问题
19、《大宪章》沉没中欧
20、自由协合党宋襄之仁
21、罗萨斯、迪亚斯之反帝新权威主义
22、贵族决斗法家，普通法决斗罗马法
23、北一辉反帝理想主义
24、英帝海盗八十年
25、欧拉、康德之乡邦，普鲁